In [33]:
import sys
# sys.path.append("../..")

import os
import pickle
import random

from tqdm import tqdm

from time import sleep

import json

In [34]:
def carregar_mapa_codigo() -> dict:
    """
    Carrega um arquivo de texto e o converte em um dicionário Python.

    O arquivo deve ter uma entrada por linha, com a chave e o valor
    separados por espaços ou uma tabulação.

    Args:

    Returns:
        dict: Um dicionário contendo os dados do arquivo.
              Retorna um dicionário vazio se o arquivo não for encontrado ou ocorrer um erro.
    """
    caminho_diretorio = os.path.join('..', '..', 'rcsc')
    os.makedirs(caminho_diretorio, exist_ok=True)
    caminho_arquivo = os.path.join(caminho_diretorio, 'mapa_codigo.txt')
     
    mapa_alimentos = {}
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            for linha in f:
                if not linha.strip():
                    continue

                partes = linha.rsplit(None, 1)
                
                if len(partes) == 2:
                    chave = partes[0].strip()  
                    valor = partes[1].strip()  
                    mapa_alimentos[chave] = valor
                    
    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado.")
        return {}
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo: {e}")
        return {}
        
    return mapa_alimentos

print(carregar_mapa_codigo())

{'Leite Integral': 'BRC0043G', 'Aveia em Flocos': 'BRC0021A', 'Maçã': 'BRC0023C', 'Banana Prata': 'BRC0011C', 'Amendoim torrado sem sal': 'BRC0021T', 'Arroz Cozido': 'BRC0209A', 'Feijão Cozido': 'BRC0091T', 'Peito de Frango Grelhado': 'BRC0230F', 'Salada de Folhas e Tomate': 'BRC0396B', 'Azeite para tempero': '-', 'Pão Francês': 'BRC0002A', 'Queijo Minas Padrão': 'BRC0056G', 'Batata Doce Cozida': 'BRC0042B', 'Ovos Cozidos': 'BRC0010J', 'Salada de Brócolis e Cenoura Cozida': 'BRC0643B,BRC0504B', 'Salada de Brócolis': 'BRC0643B', 'Salada de Cenoura': 'BRC0504B', 'Chá de Ervas': '-', 'Ovos Mexidos': 'BRC0028J', 'Azeite para preparo dos ovos': '-', 'Iogurte Natural Integral': 'BRC0011G', 'Mel': 'BRC0028K', 'Carne Moída Magra (patinho)': 'BRC0441F', 'Salada Mista (alface, tomate, pepino)': 'BRC0396B', 'Leite em Pó Integral': 'BRC0041G', 'Mandioca Cozida (Aipim)': 'BRC0908B', 'Filé de Frango Grelhado': 'BRC0230F', 'Abóbora Cozida (Jerimum)': 'BRC0166B', 'Frango Desfiado': 'BRC0230F', 'Salada

In [39]:
def carregar_alimentos_do_json() -> list:
    """
    Carrega um arquivo JSON de planos alimentares e extrai uma lista
    de todos os nomes de alimentos únicos presentes.

    Args:

    Returns:
        list: Uma lista de strings com os nomes únicos de todos os alimentos.
              Retorna uma lista vazia se o arquivo não for encontrado ou ocorrer um erro.
    """
    
    caminho_diretorio = os.path.join('..', '..', 'rcsc')
    os.makedirs(caminho_diretorio, exist_ok=True)
    caminho_arquivo = os.path.join(caminho_diretorio, 'gemini.json')
    # caminho_arquivo = os.path.join(caminho_diretorio, 'gbt.json')
    # caminho_arquivo = os.path.join(caminho_diretorio, 'deepseek.json')
    
    alimentos_unicos = set()
    
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            dados_planos = json.load(f)
            
            # O arquivo é uma lista de planos
            for plano in dados_planos:
                # Cada plano é um dicionário de dias ('1', '2', ...)
                for dia in plano.values():
                    # Cada dia é um dicionário de refeições ('Café da Manhã', ...)
                    for refeicao in dia.values():
                        # Cada refeição é uma lista de itens
                        for item in refeicao:
                            if 'alimento' in item:
                                alimentos_unicos.add(item['alimento'])
                                
    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado.")
        return []
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{caminho_arquivo}' não é um JSON válido.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return []
        
    return sorted(list(alimentos_unicos))

print(carregar_alimentos_do_json())

['Abóbora Cozida (Jerimum)', 'Abóbora Moranga Cozida', 'Amendoim torrado sem sal', 'Arroz Cozido', 'Arroz Integral Cozido', 'Atum em Lata (em água, drenado)', 'Aveia', 'Aveia em Flocos', 'Azeite (para preparo dos ovos)', 'Azeite (para preparo)', 'Azeite (para tempero)', 'Azeite para preparo dos ovos', 'Azeite para tempero', 'Banana Prata', 'Banana Prata (picada)', 'Batata Cozida', 'Batata Doce Cozida', 'Batata Inglesa Cozida', 'Biscoito Cream Cracker', 'Biscoito Maizena', 'Biscoito de Polvilho', 'Bolo de Banana (caseiro)', 'Bolo de Milho/Fubá (caseiro)', 'Café com Leite Integral', 'Café preto (sem açúcar)', 'Carne Bovina (Alcatra magra, cozida)', 'Carne Bovina (Alcatra/Maminha magra, cozida)', 'Carne Bovina (Alcatra/Maminha magra, grelhada)', 'Carne Bovina (Músculo/Paleta cozida)', 'Carne Bovina (Patrimho magro, cozido)', 'Carne Bovina Cozida (Músculo/Paleta)', 'Carne Bovina Magra (patinho, cozida)', 'Carne Moída Magra (molho)', 'Carne Moída Magra (patinho)', 'Carne Suína (Lombo magro,

In [40]:
alimentos = carregar_alimentos_do_json()
codigos = carregar_mapa_codigo()

codigos = list(codigos.keys())


faltam = []
for alimento in alimentos:
    if not alimento in  codigos:
        faltam.append(alimento)
# faltam = list(set(faltam))

print(len(faltam))
for i in range(min(len(faltam), 15)):
    print(faltam[i])


0


In [ ]:
caracteristicas = [
    'Energia', # Kcal
]

macronutrientes = [
    'Proteína',
    'Lipídios',
    # 'Carboidrato total',
    'Carboidrato disponível',
    'Fibra alimentar',
    # 'Ácidos graxos saturados',
    # 'Ácidos graxos monoinsaturados',
    # 'Ácidos graxos poliinsaturados',
    # 'Ácidos graxos trans',
    # 'Colesterol',
    # 'Proteína animal',
    # 'Proteína vegetal'
]

micronutrientes = [
    # Minerais
    'Cálcio',
    'Ferro',
    'Sódio',
    'Magnésio',
    'Fósforo',
    'Potássio',
    'Zinco',
    'Cobre',
    'Selênio',
    
    # Vitaminas
    # 'Vitamina A (RE)',
    'Vitamina A (RAE)',
    'Vitamina D',
    'Alfa-tocoferol (Vitamina E)',
    'Tiamina',
    'Riboflavina',
    'Niacina',
    'Vitamina B6',
    'Vitamina B12',
    'Vitamina C',
    'Equivalente de folato'
]